In [1]:
from ddCRP import ddCRP
from ddCRP import Priors

from surface_utilities import adjacency

import numpy as np
import nibabel as nb
from niio import loaded

In [2]:
data_file = '/Users/kristianeschenburg/Desktop/Research/Data/CorticalRegionalization/Destrieux/RestingState/285345.L.Cortical.Regionalized.RestingState.aparc.a2009s.mat'
label_file = '/Users/kristianeschenburg/Desktop/Research/Data/Labels/HCP/285345.L.CorticalAreas.fixed.32k_fs_LR.label.gii'
surf_file = '/Users/kristianeschenburg/Desktop/Research/Data/Surfaces/285345.L.inflated.32k_fs_LR.surf.gii'

In [4]:
# Load feature matrix
data = loaded.load(data_file)
[n,p] = data.shape
if n < p:
    data = data.T

In [154]:
# Get indices of non-zero data samples
indices = np.abs(data).sum(1) != 0
idx = np.arange(data.shape[0])[indices]

features = np.arange(data.shape[1])[np.var(data,axis=0) != 0]

In [3]:
# Load surface, generate adjacency list
surface = nb.load(surf_file)
vertices = surface.darrays[0].data
faces = surface.darrays[1].data

S = adjacency.SurfaceAdjacency(vertices=vertices,faces=faces)
S.generate()

# Filter adjacency list
filtered = S.filtration(filter_indices=indices,remap=True)

In [5]:
# Load initial clustering
label = loaded.load(label_file)

In [160]:
# Filter data matrix and label array
filtered_data = data[idx,:][:,features]
init_z_filtered = label[idx]

In [9]:
print(filtered_data.shape)
print(init_z_filtered.shape)

(29696, 74)
(29696,)


In [10]:
# Set hyperparameters for prior / likelihood model
alpha = 0.1
mu_bar = np.zeros((filtered_data.shape[1],))
kappa = 0.01
nu = 75
lambda_bar = np.eye(len(mu_bar)) + np.cov(filtered_data.T)

# Define prior
niw = Priors.NIW(mu0 = mu_bar, kappa0=kappa, nu0=nu, lambda0=lambda_bar)

In [217]:
import importlib
importlib.reload(ddCRP)
importlib.reload(ddCRP.subgraphs)

<module 'ddCRP.subgraphs' from '/Users/kristianeschenburg/Documents/Code/ddCRP/ddCRP/subgraphs.py'>

In [19]:
# Initialize ddCRP model
crp = ddCRP.ddCRP(alpha=alpha, mcmc_passes=300, n_clusters=180, stats_interval=5, model=niw)

In [20]:
# Fit model
crp.fit(adj_list=filtered,features=filtered_data,init_z=init_z_filtered)


Full
Step: 0 Time: -1536436467.702724 LP: -519220.74875746184 K: 1536 MaxLP: -519220.74875746184
Step: 5 Time: -1536436463.3096519 LP: -519215.5688192227 K: 1536 MaxLP: -519215.5688192227
Step: 10 Time: -1536436459.0855598 LP: -519209.31864710385 K: 1536 MaxLP: -519209.31864710385
Step: 15 Time: -1536436454.822851 LP: -519209.31864710385 K: 1536 MaxLP: -519209.31864710385
Step: 20 Time: -1536436450.5927489 LP: -519182.606576338 K: 1536 MaxLP: -519182.606576338
Step: 25 Time: -1536436446.3535419 LP: -519182.606576338 K: 1536 MaxLP: -519182.606576338
Step: 30 Time: -1536436442.116478 LP: -519182.606576338 K: 1536 MaxLP: -519182.606576338
Step: 35 Time: -1536436437.891324 LP: -519182.606576338 K: 1536 MaxLP: -519182.606576338
Step: 40 Time: -1536436433.682971 LP: -517620.88025039807 K: 1537 MaxLP: -517620.88025039807
Step: 45 Time: -1536436429.4280539 LP: -517576.98330749397 K: 1537 MaxLP: -517576.98330749397
Step: 50 Time: -1536436425.081526 LP: -517576.98330749397 K: 1537 MaxLP: -51757

KeyboardInterrupt: 

In [161]:
init_c = ddCRP.subgraphs.ClusterSpanningTrees(filtered, init_z_filtered).fit()

In [162]:
[c, G] = ddCRP.subgraphs.sparse_linkage(filtered, len(init_c), init_c)

In [211]:
len(roots)

1412

In [218]:
[c, rand_roots, roots] = ddCRP.subgraphs.ClusterSpanningTrees(filtered, init_z_filtered).fit()

Rand Root: 23638
MST: 882
Rand Root: 13779
MST: 134
Rand Root: 24035
MST: 152
Rand Root: 23764
MST: 528
Rand Root: 22738
MST: 407
Rand Root: 21738
MST: 377
Rand Root: 22507
MST: 35
Rand Root: 4227
MST: 926
Rand Root: 16284
MST: 488
Rand Root: 4182
MST: 165
Rand Root: 28141
MST: 130
Rand Root: 10365
MST: 120
Rand Root: 99
MST: 227
Rand Root: 24171
MST: 282
Rand Root: 10315
MST: 44
Rand Root: 12383
MST: 103
Rand Root: 22044
MST: 142
Rand Root: 12108
MST: 74
Rand Root: 13363
MST: 31
Rand Root: 21686
MST: 65
Rand Root: 22132
MST: 40
Rand Root: 13821
MST: 26
Rand Root: 7711
MST: 257
Rand Root: 3260
MST: 192
Rand Root: 1854
MST: 176
Rand Root: 7485
MST: 96
Rand Root: 10806
MST: 120
Rand Root: 11478
MST: 113
Rand Root: 24279
MST: 260
Rand Root: 594
MST: 68
Rand Root: 24407
MST: 65
Rand Root: 111
MST: 47
Rand Root: 1890
MST: 167
Rand Root: 2358
MST: 110
Rand Root: 2471
MST: 100
Rand Root: 2423
MST: 304
Rand Root: 10198
MST: 219
Rand Root: 2796
MST: 192
Rand Root: 1068
MST: 124
Rand Root: 10778

In [219]:
roots

array([    4,    65,    66, ..., 29350, 29444, 29497])

In [224]:
non_rands = list(set(roots).difference(set(rand_roots)))

In [248]:
a = np.asarray(G[non_rands[2],:].todense()).squeeze()
print(a.shape)

(29696,)


In [249]:
np.where(a)

(array([16397]),)

In [235]:
np.where(a.todense())

(array([0]), array([16386]))

In [166]:
[K, z, parcels] = ddCRP.subgraphs.connected_components(G)
print('Parcels: {:}'.format(K))

Parcels: 1345


In [197]:
g = np.zeros((32492,))
t = np.zeros((32492,))

c = 0
for u in np.unique(z):
    
    f = np.where(z == u)[0]
    
    if len(f) > 1:
        g[idx[f]] = u
        c += 1
        
    if len(f) == 1:
        t[idx[f]] = idx[f]
        
print('Groups: {:}'.format(c))

Groups: 172


In [202]:
len(roots)

1356

In [201]:
len(np.unique(t))

1174